In [1]:
import numpy as np
import pandas as pd
import pyvista as pv


In [2]:
input_df = pd.read_csv("data_storage/2.csv")

In [7]:
# Get the reference halo coordinates
ref_halo = input_df[input_df['fof_halo_tag'] == 251375070].iloc[0]

# Calculate Euclidean distance for all halos
input_df['distance'] = ((input_df['fof_halo_center_x'] - ref_halo['fof_halo_center_x'])**2 +
                        (input_df['fof_halo_center_y'] - ref_halo['fof_halo_center_y'])**2 +
                        (input_df['fof_halo_center_z'] - ref_halo['fof_halo_center_z'])**2).pow(0.5)

# Sort by distance and select the 10 closest halos (excluding the reference halo)
result_df = input_df[input_df['fof_halo_tag'] != '251375070'].nsmallest(10, 'distance')
result_df

,fof_halo_tag,fof_halo_center_x,fof_halo_center_y,fof_halo_center_z,distance
0,251375070.0,120.370285,62.418175,61.067566,0.000000
225448,257621504.0,120.634964,61.661469,60.639606,0.908740
216976,247737798.0,121.082611,61.841316,60.971027,0.921680
217770,248726076.0,120.846283,61.665443,61.370796,0.940812
221888,253467068.0,120.734375,61.692284,61.571365,0.955664
231657,265026024.0,120.875916,61.895432,61.695572,0.960892
226815,259228238.0,120.713974,61.644142,61.523071,0.961631
234342,268173774.0,121.187424,61.820312,61.233467,1.026001
225452,257625634.0,119.937660,63.341309,60.815285,1.050231
233366,267068976.0,119.875992,63.234344,60.543144,1.088796


In [4]:
reference_halo = input_df[input_df['fof_halo_tag'] == 251375070][['fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z']].iloc[0]
distances = ((input_df[['fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z']] - reference_halo) ** 2).sum(axis=1) ** 0.5
result_df = input_df.loc[distances.nsmallest(10).index]
result_df

,fof_halo_tag,fof_halo_center_x,fof_halo_center_y,fof_halo_center_z
0,251375070.0,120.370285,62.418175,61.067566
225448,257621504.0,120.634964,61.661469,60.639606
216976,247737798.0,121.082611,61.841316,60.971027
217770,248726076.0,120.846283,61.665443,61.370796
221888,253467068.0,120.734375,61.692284,61.571365
231657,265026024.0,120.875916,61.895432,61.695572
226815,259228238.0,120.713974,61.644142,61.523071
234342,268173774.0,121.187424,61.820312,61.233467
225452,257625634.0,119.937660,63.341309,60.815285
233366,267068976.0,119.875992,63.234344,60.543144


In [7]:
input_df = pd.read_csv("./data_storage/120_1.csv")
input_df

,fof_halo_tag,sod_halo_center_x,sod_halo_center_y,sod_halo_center_z,sod_halo_mass,distance
0,251375070.0,120.370285,62.418175,61.067566,5.236229e+14,0.0


In [ ]:
# Extract coordinates and create PolyData object
points = input_df[['sod_halo_center_x', 'sod_halo_center_y', 'sod_halo_center_z']].to_numpy()
pdata = pv.PolyData(points)

# Add scalar fields
pdata['mass'] = input_df['sod_halo_mass'].to_numpy()
pdata['distance'] = input_df['distance'].to_numpy()

# Identify the largest halo (first row, as it's sorted)
largest_halo = pdata.points[0]

# Create a sphere to represent the largest halo
sphere = pv.Sphere(radius=0.5, center=largest_halo)

# Create a plotter
plotter = pv.Plotter(off_screen=True)

# Add the points (halos) to the plot, colored by mass
plotter.add_mesh(pdata, scalars='mass', cmap='viridis', point_size=10, render_points_as_spheres=True)

# Add the largest halo sphere
plotter.add_mesh(sphere, color='red')

# Set up the camera and other properties
plotter.camera_position = 'xy'
plotter.camera.zoom(1.5)
plotter.show_bounds(grid='front', location='outer', all_edges=True)

# # Save the plot as a VTK file
# plotter.export_vtksz('data_storage/halo_visualization.vtk')

# # Also save as a static image for quick viewing
# plotter.screenshot('data_storage/halo_visualization.png')

# # Close the plotter
# plotter.close()